# Video Scraper

Download youtube videos and frame cutter

Created by [Artem Konevskikh](https://github.com/artem-konevskikh)

In [ ]:
#@title Install

# !pip install --upgrade pytube
!python -m pip install git+https://github.com/kinshuk-h/pytube
!apt -y install ffmpeg

In [ ]:
#@title Initialize
import os
from tqdm.notebook import tqdm
from pytube import YouTube, Playlist, extract
from pytube.cli import on_progress

def download_youtube(url, output_path):
 
    """ Instantiates YouTube class and downloads selected video.  Uses Built-in
    pytube.cli function on_progress to show a DOS style progress bar. """
    yt = YouTube(url, on_progress_callback=on_progress)
    st = yt.streams.get_highest_resolution()
    id=extract.video_id(url)
    filename = f"{id}.{st.default_filename.split('.')[-1]}"
  
    # following line displays title and number of times video has been viewed. 
    print(f'Downloading: {url}')
  
    # creates download and downloads to subdirectory called 'downloads'
    st.download(output_path=output_path, filename=filename)
 
    # displays message verifying download is complete, and resets color scheme 
    # back to original color scheme.
    print(f'Video saved to:  {output_path}/{filename}')

In [ ]:
#@title Mount Google Drive
#@markdown Mount Google Drive to save search results.

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

if not os.path.exists('/content/drive/MyDrive/scraped-videos'):
  os.makedirs('/content/drive/MyDrive/scraped-videos')

Mounted at /content/drive


## Download videos

In [ ]:
youtube_links = [
"https://www.youtube.com/watch?v=i9T727tz7FA",
"https://www.youtube.com/watch?v=BaB46NSwvNA"
]

In [ ]:
#@title Download
#@markdown **Folder name**<br/>
#@markdown Folder will be created inside `scraped-videos` folder on your Google Drive. Please, try not to use spaces in folderr names and better name it in CamelCase or snake_case style
output_folder = 'Mycelium' #@param {type: "string"}
output_path = f'/content/drive/MyDrive/scraped-videos/{output_folder}/'
!mkdir $output_path
for url in tqdm(youtube_links):
  download_youtube(url, output_path)

## Slice frames

In [ ]:
#@title Slice
#@markdown Video 
input_file = '/content/drive/MyDrive/scraped-videos/Mycelium/BaB46NSwvNA.mp4' #@param {type: "string"}
#@markdown Folder to save frames 
frames_folder = '' #@param {type: "string"}
#@markdown Frames per second
fps=5 #@param {type: "integer"}
#@markdown Prefix 
prefix = '' #@param {type: "string"}
#@markdown File format 
ext = 'png' #@param ["png","jpg"]

if frames_folder == '':
  frames_folder='/content/frames'
if prefix:
  out_frame = f'{frames_folder}/{prefix}-frame%08d.{ext}'
else:
  out_frame = f'{frames_folder}/frame%08d.{ext}'
!mkdir $frames_folder
!ffmpeg -i $input_file -vf fps=$fps $out_frame